In [123]:
from itertools import chain 

In [211]:
class strand:
    def __init__(self, alpha, prev = None, next = None):
        self.alpha = alpha 
        self.stack = 1
        self.prev_crossing = prev
        self.next_crossing = next
    
    def idempotence(self):
        if self.stack > 1 :
            self.stack = 1
        elif self.stack < 0:
            self.stack = 0
            
    def idempotency_after_move(self):
        for strand in {self, self.next_crossing.next_strand, self.prev_crossing.prev_strand, 
                       self.next_crossing.pair_crossing.prev_strand, self.next_crossing.pair_crossing.next_strand,
                      self.prev_crossing.pair_crossing.prev_strand, self.prev_crossing.pair_crossing.next_strand}:
            strand.idempotence()
        return True
    
    def total_move_to_next(self, accumulate = False):
        if self.stack < 1 or self.next_crossing.pair_crossing.prev_strand.stack < 1:
            return False
        if not accumulate:
            self.stack -=1
            self.next_crossing.pair_crossing.prev_strand.stack -= 1

        if self.next_crossing.next_strand.stack == 0:
            self.next_crossing.next_strand.alpha = self.next_crossing.pair_crossing.prev_strand.alpha
            self.next_crossing.next_strand.stack = 1
        else:
            if self.next_crossing.next_strand.alpha != self.next_crossing.pair_crossing.prev_strand.alpha:
                return False
            self.next_crossing.next_strand.stack += 1
            
        if self.next_crossing.pair_crossing.next_strand.stack == 0:
            self.next_crossing.pair_crossing.next_strand.alpha = self.alpha
            self.next_crossing.pair_crossing.next_strand.stack = 1
        else:
            if self.next_crossing.pair_crossing.next_strand.alpha != self.alpha:
                return False
            self.next_crossing.pair_crossing.next_strand.stack += 1
        self.idempotency_after_move()
        return True
    
    def __str__(self):
        if self.stack > 0:
            return (str(self.alpha) +':'+ str(self.stack))
        else:
            return ''
        
class crossing:
    def __init__(self, number, sign, pair = None, prev = None, next = None):
        self.number = number
        self.sign = sign # 0 : upper, 1 : lower
        self.pair_crossing = pair
        self.prev_strand = prev
        self.next_strand = next
    
    def __str__(self):
        return '('+str(self.number)+('+' if self.sign else '-')+')'
    
class link:
    def __init__(self, gauss_codes):
        self.checked_number = dict({})
        self.idxs = list([])
        self.ordering = list([])
        self.is_link = self.knotify(gauss_code)
        
        
    def knotify(self, gauss_codes):
        link = True
        # initialize
        # Check there are pairs
        flatten_code = list(chain(*gauss_codes))
        for crossing in flatten_code:
            if crossing.number not in self.checked_number:
                for elt in flatten_code:
                    if elt.number == crossing.number:
                        if elt == crossing:
                            continue
                        else:
                            if elt.sign == crossing.sign:
                                link = False
                                return link
                            elif elt.number not in self.checked_number: 
                                crossing.pair_crossing = elt
                                elt.pair_crossing = crossing
                                self.checked_number[crossing.number] = crossing
                            else:
                                link = False
        
        link = (len(flatten_code)/2 == len(self.checked_number))
        if not link:
            return False
        # make strand-crossing-complex
        for gauss_code in gauss_codes:
            current_node = strand('X')
            idx = current_node
            self.idxs += [idx]
            for crossing in gauss_code:
                current_node.next_crossing = crossing
                crossing.prev_strand = current_node
                crossing.next_strand = strand('X')
                current_node = crossing.next_strand
                current_node.prev_crossing = crossing
            idx.prev_crossing = gauss_code[-1]
            gauss_code[-1].next_strand = idx
        # accumulation algorithm
        while len(self.ordering) < len(self.checked_number) and link:
            for number in self.checked_number:
                if self.checked_number[number].prev_strand.total_move_to_next():
                    self.ordering += [number]
                    link = True
                    break    
                link = False
        if not link:
            return False
        # label strands
        index = 0
        for idx in self.idxs:
            current_node = idx
            while True:
                if current_node.stack != 0:
                    current_node.alpha = 'S_'+str(index)
                    index += 1
                current_node = current_node.next_crossing.next_strand
                if current_node in self.idxs:
                    break
        # Check labelling is well
        for number in self.ordering:
            rslt = self.checked_number[number].prev_strand.total_move_to_next(True)
            print(self)
            link = link and rslt
        print(self)
        return link
            
    def __str__(self):
        rslt = ''
        for idx in self.idxs:
            flag = True
            current_node = idx
            rslt += str(current_node)
            while flag:
                current_node = current_node.next_crossing
                rslt += str(current_node)
                current_node = current_node.next_strand
                if current_node == idx:
                    flag = False
                else:
                    rslt += str(current_node)
            rslt += '/'
        return rslt
            

In [212]:
gauss_code = [[crossing(1, 1), crossing(2,0), crossing(3, 1), crossing(1, 0), crossing(3,0), crossing(2,1)]]
print(link(gauss_code).is_link)
gauss_code = [[crossing(1, 1), crossing(2,0), crossing(3, 1), crossing(1, 0), crossing(2,1), crossing(3,0)]]
print(link(gauss_code).is_link)
gauss_code = [[crossing(1,1), crossing(2,1), crossing(1,0), crossing(2,0)]]
print(link(gauss_code).is_link)
gauss_code = [[crossing(1,1), crossing(2,1)],[crossing(1,0), crossing(2,0)]]
print(link(gauss_code).is_link)

S_0:1(1+)S_1:1(2-)(3+)S_1:1(1-)S_0:1(3-)S_2:1(2+)/
S_0:1(1+)S_1:1(2-)S_2:1(3+)S_1:1(1-)S_0:1(3-)S_2:1(2+)/
S_0:1(1+)S_1:1(2-)S_2:1(3+)S_1:1(1-)S_0:1(3-)S_2:1(2+)/
S_0:1(1+)S_1:1(2-)S_2:1(3+)S_1:1(1-)S_0:1(3-)S_2:1(2+)/
False
S_0:1(1+)S_1:1(2-)(3+)S_1:1(1-)S_0:1(2+)(3-)/
S_0:1(1+)S_1:1(2-)S_0:1(3+)S_1:1(1-)S_0:1(2+)S_1:1(3-)/
S_0:1(1+)S_1:1(2-)S_0:1(3+)S_1:1(1-)S_0:1(2+)S_1:1(3-)/
S_0:1(1+)S_1:1(2-)S_0:1(3+)S_1:1(1-)S_0:1(2+)S_1:1(3-)/
True
S_0:1(1+)S_1:1(2+)S_1:1(1-)S_0:1(2-)/
S_0:1(1+)S_1:1(2+)S_1:1(1-)S_0:1(2-)/
S_0:1(1+)S_1:1(2+)S_1:1(1-)S_0:1(2-)/
False
S_0:1(1+)S_1:1(2+)/S_1:1(1-)S_0:1(2-)/
S_0:1(1+)S_1:1(2+)/S_1:1(1-)S_0:1(2-)/
S_0:1(1+)S_1:1(2+)/S_1:1(1-)S_0:1(2-)/
True
